# Nurse Scheduling
### Demo from  https://github.com/dwave-examples/nurse-scheduling

This is a demo of a nurse scheduling model developed by Ikeda, Nakamura
and Humble (INH).

The nurse scheduling problem seeks to find an optimal assignment for a group of
nurses, under constraints of scheduling and personnel. INH developed a model
which is a simplified representation of a real-world nursing facility.

In the general nurse scheduling problem, there are three types of constraints,
which are mentioned here to provide background for INH's constraints.
These types of constraints, in the general problem, are:

1) Both upper and lower limits on the number of breaks.
2) The number of nurses on duty for each shift slot.
3) For each individual nurse, upper and lower limits on the time interval
   between days of duty.

These three types of constraints combine to ensure sufficient nurses
on duty at all times, without overworking any particular nurse.

INH formulated a QUBO from a simplification of these constraints, discussed
below, that tries to achieve reasonable results for nurse scheduling.

INH's three types of constraints are:

1) "hard shift" constraint: requires that at least one nurse is assigned for
   each working day.

2) "hard nurse" constraint: requires that no nurse works two or more consecutive
   days.

3) "soft nurse" constraint: promotes that all nurses should have roughly
   even work schedules.

This demo seeks to obtain reasonable results for a nurse schedule, based on
INH's model. Our implementation attempts to find a schedule for a number
`n_nurses` of nurses and a number `n_days` of days that satisfies the following
conditions:

* One, and only one, nurse has been assigned to each day (hard shift constraint)
* No nurse works two days in a row (hard nurse constraint)
* The nurses should work the same number of days

Running the demo results in the following output, at the command-line:

```bash
Building binary quadratic model...

Sending problem to hybrid sampler...

Building schedule and checking constraints...

	Hard shift constraint: Satisfied
	Hard nurse constraint: Satisfied
	Soft nurse constraint: Unsatisfied

Schedule:

Nurse  2         X        X     X     X    
Nurse  1      X        X           X     X 
Nurse  0   X        X        X             
           0  1  2  3  4  5  6  7  8  9  10 

Schedule saved as schedule.png.
```

The results show the following:

* One, and only one, nurse has been assigned to each day
* No nurse works two days in a row
* Two nurses work 4 days, and one works three days. Because two nurses work one
  extra day each, the soft nurse constraint energy is unsatisfied.

## Code Specifics

Some notes on the code:

* We use a two-dimensional QUBO matrix, Q[`i`, `j`], in which both indices `i`
  and `j` are composite indices. Each composite index is used to represent the
  combinations of the variables `nurse` and `day`. The (`nurse`, `day`) tuples
  are placed into the one-dimensional index in the following order, where
  `nurse` is first index, and `day` is second index, in the tuples:

  (0, 0) (0, 1) (0, 2)... (0, D) (1, 0) (1, 1)... (1, D)

* The methods `get_index` and `get_nurse_and_day` are used to convert back and
  forth between (`nurse`, `day`) tuples and the composite indices.

* The three constraint sums are separated out in order to be able to confirm the
  individual effects manually. For example, if a nurse was assigned to two
  successive days, the hard nurse constraint sum would be nonzero.

* We have not yet confirmed Ikeda's results with reverse annealing

In [ ]:
from dwave.system import LeapHybridSampler
from dimod import BinaryQuadraticModel
from collections import defaultdict
from copy import deepcopy
import matplotlib

try:
    import matplotlib.pyplot as plt
    from matplotlib.patches import Rectangle
except ImportError:
    matplotlib.use("agg")
    import matplotlib.pyplot as plt
    from matplotlib.patches import Rectangle

In [ ]:
# Overall model variables: problem size
# binary variable q_nd is the assignment of nurse n to day d
n_nurses = 3      # count nurses n = 1 ... n_nurses
n_days = 11       # count scheduling days as d = 1 ... n_days
size = n_days * n_nurses

# Parameters for hard nurse constraint
# a is a positive correlation coefficient for implementing the hard nurse
# constraint - value provided by Ikeda, Nakamura, Humble
a = 3.5

# Parameters for hard shift constraint
# Hard shift constraint: at least one nurse working every day
# Lagrange parameter, for hard shift constraint, on workforce and effort
lagrange_hard_shift = 1.3
workforce = 1     # Workforce function W(d) - set to a constant for now
effort = 1        # Effort function E(n) - set to a constant for now

# Parameters for soft nurse constraint
# Soft nurse constraint: all nurses should have approximately even work
#                        schedules
# Lagrange parameter, for shift constraints, on work days is called gamma
# in the paper
# Minimum duty days 'min_duty_days' - the number of work days that each
# nurse wants
# to be scheduled. At present, each will do the minimum on average.
# The parameter gamma's value suggested by Ikeda, Nakamura, Humble
lagrange_soft_nurse = 0.3      # Lagrange parameter for soft nurse, gamma
preference = 1                 # preference function - constant for now
min_duty_days = int(n_days/n_nurses)


# Find composite index into 1D list for (nurse_index, day_index)
def get_index(nurse_index, day_index):
    return nurse_index * n_days + day_index


# Inverse of get_index - given a composite index in a 1D list, return the
# nurse_index and day_index
def get_nurse_and_day(index):
    nurse_index, day_index = divmod(index, n_days)
    return nurse_index, day_index

print("\nBuilding binary quadratic model...")
# Hard nurse constraint: no nurse works two consecutive days
# It does not have Lagrange parameter - instead, J matrix
# symmetric, real-valued interaction matrix J, whereas all terms are
# a or zero.
# composite indices i(n, d) and j(n, d) as functions of n and d
# J_i(n,d)j(n,d+1) = a and 0 otherwise.
J = defaultdict(int)
for nurse in range(n_nurses):
    for day in range(n_days - 1):
        nurse_day_1 = get_index(nurse, day)
        nurse_day_2 = get_index(nurse, day+1)
        J[nurse_day_1, nurse_day_2] = a

# Q matrix assign the cost term, the J matrix
Q = deepcopy(J)

# Hard shift constraint: at least one nurse working every day
# The sum is over each day.
# This constraint tries to make (effort * sum(q_i)) equal to workforce,
# which is set to a constant in this implementation, so that one nurse
# is working each day.
# Overall hard shift constraint:
# lagrange_hard_shift * sum_d ((sum_n(effort * q_i(n,d)) - workforce) ** 2)
#
# with constant effort and constant workforce:
# = lagrange_hard_shift * sum_d ( effort * sum_n q_i(n,d) - workforce ) ** 2
# = lagrange_hard_shift * sum_d [ effort ** 2 * (sum_n q_i(n,d) ** 2)
#                              - 2 effort * workforce * sum_n q_i(n,d)
#                              + workforce ** 2 ]
# The constant term is moved to the offset, below, right before we solve
# the QUBO
#
# Expanding and merging the terms ( m is another sum over n ):
# lagrange_hard_shift * (effort ** 2 - 2 effort * workforce) *
# sum_d sum_n q_i(n,d)
# + lagrange_hard_shift * effort ** 2 * sum_d sum_m sum_n q_i(n,d) q_j(m, d) #

# Diagonal terms in hard shift constraint, without the workforce**2 term
for nurse in range(n_nurses):
    for day in range(n_days):
        ind = get_index(nurse, day)
        Q[ind, ind] += lagrange_hard_shift * (effort ** 2 - (2 * workforce * effort))

# Off-diagonal terms in hard shift constraint
# Include only the same day, across nurses
for day in range(n_days):
    for nurse1 in range(n_nurses):
        for nurse2 in range(nurse1 + 1, n_nurses):

            ind1 = get_index(nurse1, day)
            ind2 = get_index(nurse2, day)
            Q[ind1, ind2] += 2 * lagrange_hard_shift * effort ** 2

# Soft nurse constraint: all nurses should have approximately even work
#                        schedules
# This constraint tries to make preference * sum(q_i) equal to min_duty_days,
# so that the nurses have the same number of days. The sum of the q_i,
# over the number of days, is each nurse's number of days worked in the
# schedule.
# Overall soft nurse constraint:
# lagrange_soft_nurse * sum_n ((sum_d(preference * q_i(n,d)) - min_duty_days) ** 2)
# with constant preference and constant min_duty_days:
# = lagrange_soft_nurse * sum_n ( preference * sum_d q_i(n,d) - min_duty_days ) ** 2
# = lagrange_soft_nurse * sum_n [ preference ** 2 * (sum_d q_i(n,d) ** 2)
#                              - 2 preference * min_duty_days * sum_d q_i(n,d)
#                              + min_duty_days ** 2 ]
# The constant term is moved to the offset, below, right before we solve
# the QUBO
#
# The square of the the sum_d term becomes:
# Expanding and merging the terms (d1 and d2 are sums over d):
# = lagrange_soft_nurse * (preference ** 2 - 2 preference * min_duty_days) * sum_n sum_d q_i(n,d)
# + lagrange_soft_nurse * preference ** 2 * sum_n sum_d1 sum_d2 q_i(n,d1)
#                      * q_j(n, d2)

# Diagonal terms in soft nurse constraint, without the min_duty_days**2 term
for nurse in range(n_nurses):
    for day in range(n_days):
        ind = get_index(nurse, day)
        Q[ind, ind] += lagrange_soft_nurse * (preference ** 2 - (2 * min_duty_days * preference))

# Off-diagonal terms in soft nurse constraint
# Include only the same nurse, across days
for nurse in range(n_nurses):
    for day1 in range(n_days):
        for day2 in range(day1 + 1, n_days):

            ind1 = get_index(nurse, day1)
            ind2 = get_index(nurse, day2)
            Q[ind1, ind2] += 2 * lagrange_soft_nurse * preference ** 2

# Solve the problem, and use the offset to scale the energy
e_offset = (lagrange_hard_shift * n_days * workforce ** 2) + (lagrange_soft_nurse * n_nurses * min_duty_days ** 2)
bqm = BinaryQuadraticModel.from_qubo(Q, offset=e_offset)

print("\nSending problem to hybrid sampler...")
sampler = LeapHybridSampler()
results = sampler.sample(bqm, label='Example - Nurse Scheduling')

# Get the results
smpl = results.first.sample

# Graphics
print("\nBuilding schedule and checking constraints...\n")
sched = [get_nurse_and_day(j) for j in range(size) if smpl[j] == 1]

In [ ]:
def check_hard_shift_constraint(sched, n_days):

    satisfied = [False] * n_days
    for _, day in sched:
        satisfied[day] = True

    if all(satisfied):
        return "Satisfied"
    else:
        return "Unsatisfied"

def check_hard_nurse_constraint(sched, n_nurses):

    satisfied = [True] * n_nurses
    for nurse, day in sched:
        if ((nurse, day+1) in sched) or ((nurse, day-1) in sched):
            satisfied[nurse] = False
    if all(satisfied):
        return "Satisfied"
    else:
        return "Unsatisfied"

def check_soft_nurse_constraint(sched, n_nurses):

    num_shifts = [0] * n_nurses
    for nurse, _ in sched:
        num_shifts[nurse] += 1

    if num_shifts.count(num_shifts[0]) == len(num_shifts):
        return "Satisfied"
    else:
        return "Unsatisfied"

print("\tHard shift constraint:", check_hard_shift_constraint(sched, n_days))
print("\tHard nurse constraint:", check_hard_nurse_constraint(sched, n_nurses))
print("\tSoft nurse constraint:", check_soft_nurse_constraint(sched, n_nurses))

In [ ]:
# Save image of schedule
x,y = zip(*sched)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(y, x)
width = 1
height = 1
for a_y, a_x in sched:
    if a_y == 0:
        ax.add_patch(Rectangle(
            xy=(a_x-width/2, a_y-height/2) ,width=width, height=height,
            linewidth=1, color='blue', fill=True))
    elif a_y == 1:
        ax.add_patch(Rectangle(
            xy=(a_x-width/2, a_y-height/2) ,width=width, height=height,
            linewidth=1, color='red', fill=True))
    else:
        ax.add_patch(Rectangle(
            xy=(a_x-width/2, a_y-height/2) ,width=width, height=height,
            linewidth=1, color='green', fill=True))
ax.axis('equal')
ax.set_xticks(range(n_days))
ax.set_yticks(range(n_nurses))
ax.set_xlabel("Shifts")
ax.set_ylabel("Nurses")
plt.savefig("schedule.png")

# Print schedule to command-line
print("\nSchedule:\n")
for n in range(n_nurses-1, -1, -1):
    str_row = ""
    for d in range(n_days):
        outcome = "X" if (n, d) in sched else " "
        if d > 9:
            outcome += " "
        str_row += "  " + outcome
    print("Nurse ", n, str_row)

str_header_for_output = " " * 11
str_header_for_output += "  ".join(map(str, range(n_days)))
print(str_header_for_output, "\n")

print("Schedule saved as schedule.png.")

## References

Ikeda, K., Nakamura, Y. & Humble, T.S. Application of Quantum Annealing to Nurse
Scheduling Problem. Sci Rep 9, 12837 (2019).
https://doi.org/10.1038/s41598-019-49172-3